<a href="https://colab.research.google.com/github/EvenXu2022/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-24 17:15:19--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.56MB/s    in 0.2s    

2022-04-24 17:15:19 (5.56 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
# step 1: create the vine_tale dataframe
from pyspark.sql.functions import col
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"]).dropna()
vine_votes_df = vine_df.filter(col("total_votes") >= 20)
vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|
| RLV4XQ49EBYYU|          1|           56|         65|   N|                Y|
| RPBPAJKFD0T26|          5|           63|         63|   N|                Y|
|R1IHO8C56KBN6C|          1|           61|         64|   N|     

In [11]:
# Step 2: filter percentage of helpful/total votes >=50%

percent_votes_df = vine_votes_df.withColumn('percent_votes',col('helpful_votes')/col('total_votes')).alias('percent_votes').filter(col("percent_votes") >= 0.5)
percent_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|               1.0|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|0.6918238993710691|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|               1.0|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|               1.0|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|               1.0|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|               1.0|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|0.8297872340425532|
| RLV4XQ49EBYYU|          1|  

In [12]:
# Step 3: filter where Vine == "Y"

vine_paid_df = percent_votes_df.filter(col("vine")=="Y")
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3JIGR122X50ZV|          5|         2280|       2314|   Y|                N|0.9853068280034573|
|R3KV8P8WLD2KKC|          5|          263|        286|   Y|                N|0.9195804195804196|
|R2LFY2ZPJ3J7RH|          4|           36|         38|   Y|                N|0.9473684210526315|
|R3P01OXMA0U0CA|          5|          114|        117|   Y|                N|0.9743589743589743|
|R2SCMXBD9OQP8Z|          2|           22|         26|   Y|                N|0.8461538461538461|
|R2FGS9ECOK5IQN|          5|           22|         23|   Y|                N|0.9565217391304348|
| R1V9CO41XS6QY|          2|           56|         58|   Y|                N|0.9655172413793104|
|R1LEP1JGXRY2YT|          2|  

In [13]:
# Step 4: filter where Vine == "N"

vine_unpaid_df = percent_votes_df.filter(col("vine")=="N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|               1.0|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|0.6918238993710691|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|               1.0|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|               1.0|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|               1.0|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|               1.0|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|0.8297872340425532|
| RLV4XQ49EBYYU|          1|  

In [20]:
from pyspark.sql.functions import when,count,lit

analysis_df = percent_votes_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_review"),
    count(when(col("star_rating")==5,True)).alias("Total"),
   (count(when(col("star_rating")==5,True))/count(col("vine"))*100).alias("5_star_percent"))

analysis_df.show()

+----+------------+-----+------------------+
|vine|Total_review|Total|    5_star_percent|
+----+------------+-----+------------------+
|   Y|         969|  430|44.375644994840044|
|   N|       43745|19233| 43.96616756200709|
+----+------------+-----+------------------+

